# HW 3 Fairness and Bias Analysis in Machine Learning (ML)
- Can "Run All Cells"

# Imports + Load Data

In [1]:
import pandas as pd
import numpy as np
import pingouin as pg
from scipy import stats

In [2]:
BASE = "/Users/brinkley97/Documents/development/"
CLASS_PATH = "classes/csci_535_multimodal_probabilistic_learning/"
FBA_DATASET_PATH = "datasets/"
FBA_csv = BASE + CLASS_PATH + FBA_DATASET_PATH + "data.csv"
# FBA_csv

In [3]:
def load_data(file):
    original_data = pd.read_csv(file)
    # original_data = pd.DataFrame(file)
    copy_of_data = original_data.copy()
    return copy_of_data

In [4]:
dataset_copy = load_data(FBA_csv)
dataset_copy

,Filename,Image #,Gender,Race,Confident,Egotistic,Intelligent,Kind,Responsible,Trustworthy,Aggressive,Caring,Emotional,Friendly,Sociable,Qualified
0,Google_1_Danielle Goble_5_oval.jpg,1,1,1,5.67,3.67,5.60,5.00,5.80,5.33,4.80,4.67,3.67,4.33,4.67,0
1,Google_1_Phillip Owensby_9_oval.jpg,2,0,1,6.53,2.20,6.60,7.00,7.47,6.73,3.13,6.93,5.80,6.73,6.60,1
2,Google_1_Douglas Ziegler_3_oval.jpg,3,1,1,6.07,3.60,6.47,6.47,6.67,6.40,2.40,6.07,5.14,6.67,6.67,0
3,Google_1_Donald Sauls_11_oval.jpg,4,1,2,4.80,4.60,4.47,3.80,4.27,3.87,5.80,3.53,2.80,2.93,3.27,0
4,Google_1_Eric Harman_5_oval.jpg,5,1,6,6.67,5.40,6.33,4.87,5.87,4.67,4.27,4.07,4.80,5.20,5.33,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2217,Google_1_Alison Summerville_3_oval.jpg,2218,0,5,6.40,3.73,6.27,5.53,5.87,6.07,3.87,6.13,4.33,5.53,5.27,0
2218,Google_1_Douglas Yun_3_oval.jpg,2219,0,3,5.47,5.73,5.67,4.40,5.67,4.07,5.33,4.60,4.67,4.40,4.13,0
2219,Google_1_Wayne Barrios_9_oval.jpg,2220,1,1,6.67,4.20,5.60,5.93,5.67,5.53,3.80,5.87,4.80,6.53,6.87,1
2220,Google_1_Jennie Kissinger_15_oval.jpg,2221,0,1,6.27,3.93,6.07,5.93,5.60,6.60,3.80,5.53,5.07,6.67,6.13,1


In [7]:
traits_df = dataset_copy.drop(['Filename', 'Image #', 'Gender', 'Race', 'Qualified'], axis=1)
traits_df

,Confident,Egotistic,Intelligent,Kind,Responsible,Trustworthy,Aggressive,Caring,Emotional,Friendly,Sociable
0,5.67,3.67,5.60,5.00,5.80,5.33,4.80,4.67,3.67,4.33,4.67
1,6.53,2.20,6.60,7.00,7.47,6.73,3.13,6.93,5.80,6.73,6.60
2,6.07,3.60,6.47,6.47,6.67,6.40,2.40,6.07,5.14,6.67,6.67
3,4.80,4.60,4.47,3.80,4.27,3.87,5.80,3.53,2.80,2.93,3.27
4,6.67,5.40,6.33,4.87,5.87,4.67,4.27,4.07,4.80,5.20,5.33
...,...,...,...,...,...,...,...,...,...,...,...
2217,6.40,3.73,6.27,5.53,5.87,6.07,3.87,6.13,4.33,5.53,5.27
2218,5.47,5.73,5.67,4.40,5.67,4.07,5.33,4.60,4.67,4.40,4.13
2219,6.67,4.20,5.60,5.93,5.67,5.53,3.80,5.87,4.80,6.53,6.87
2220,6.27,3.93,6.07,5.93,5.60,6.60,3.80,5.53,5.07,6.67,6.13


In [8]:
traits = traits_df.columns
traits

Index(['Confident', 'Egotistic', 'Intelligent', 'Kind', 'Responsible',
       'Trustworthy', 'Aggressive', 'Caring', 'Emotional', 'Friendly',
       'Sociable'],
      dtype='object')

# TODOs

1. Analysis of Variance (ANOVA)
2. T-Test
3. Rank the top 5 most significant biases you have found (from either race or gender)
4. Plot the histograms for the most significant biases

## 1. Analysis of Variance (ANOVA)

- To compare 11 traits across 7 races
- IV: race = [black, white, east asian, south asian, hispanic, middle eastern, other]
- DV: psychological traits = [confident, egotistic, intelligent, kind, responsible, trustworthy, aggressive, caring, emotional, friendly, sociable]
- Outcome: p-value for each trait that'll represent the bias...

In [107]:
race_series = dataset_copy.loc[0:, 'Race']
# race_series 

In [108]:
race_and_traits_df = pd.concat([race_series, traits_df], axis=1)
race_and_traits_df

,Race,Confident,Egotistic,Intelligent,Kind,Responsible,Trustworthy,Aggressive,Caring,Emotional,Friendly,Sociable
0,1,5.67,3.67,5.60,5.00,5.80,5.33,4.80,4.67,3.67,4.33,4.67
1,1,6.53,2.20,6.60,7.00,7.47,6.73,3.13,6.93,5.80,6.73,6.60
2,1,6.07,3.60,6.47,6.47,6.67,6.40,2.40,6.07,5.14,6.67,6.67
3,2,4.80,4.60,4.47,3.80,4.27,3.87,5.80,3.53,2.80,2.93,3.27
4,6,6.67,5.40,6.33,4.87,5.87,4.67,4.27,4.07,4.80,5.20,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...
2217,5,6.40,3.73,6.27,5.53,5.87,6.07,3.87,6.13,4.33,5.53,5.27
2218,3,5.47,5.73,5.67,4.40,5.67,4.07,5.33,4.60,4.67,4.40,4.13
2219,1,6.67,4.20,5.60,5.93,5.67,5.53,3.80,5.87,4.80,6.53,6.87
2220,1,6.27,3.93,6.07,5.93,5.60,6.60,3.80,5.53,5.07,6.67,6.13


In [109]:
# mean_of_traits_on_race = race_and_traits_df.mean()
# mean_of_traits_on_race

In [110]:
store_aov_with_trait = []
stort_trait = []
aov_p_values = []

for trait in range(len(traits)):
    specific_trait = traits[trait]
    print(specific_trait)
    stort_trait.append(specific_trait)
    # aov = pg.anova(dv=specific_trait, between='Race', data=race_and_traits_df, detailed=True)
    print(aov)
    aov_p_values.append(aov)
    print()

Confident
   Source           SS    DF        MS         F     p-unc       np2
0    Race     4.676867     6  0.779478  1.158257  0.325971  0.003128
1  Within  1490.639710  2215  0.672975       NaN       NaN       NaN

Egotistic
   Source           SS    DF        MS         F     p-unc       np2
0    Race    30.545907     6  5.090984  5.746486  0.000006  0.015328
1  Within  1962.334988  2215  0.885930       NaN       NaN       NaN

Intelligent
   Source           SS    DF        MS         F         p-unc       np2
0    Race    43.803961     6  7.300660  13.74459  2.175962e-15  0.035895
1  Within  1176.532882  2215  0.531166       NaN           NaN       NaN

Kind
   Source          SS    DF        MS         F     p-unc       np2
0    Race    30.63207     6  5.105345  4.358183  0.000221  0.011668
1  Within  2594.73703  2215  1.171439       NaN       NaN       NaN

Responsible
   Source           SS    DF        MS          F         p-unc       np2
0    Race    45.293319     6  7.5488

In [112]:
# list(race_and_traits_df['Confident'])

[5.67,
 6.53,
 6.07,
 4.8,
 6.67,
 6.5,
 6.2,
 4.93,
 6.47,
 5.27,
 5.93,
 6.47,
 5.33,
 3.67,
 5.93,
 5.67,
 5.13,
 7.13,
 6.87,
 4.87,
 4.13,
 5.36,
 7.13,
 6.2,
 4.21,
 6.53,
 7.31,
 6.53,
 5.5,
 5.2,
 6.47,
 5.07,
 7.53,
 5.0,
 6.21,
 4.8,
 6.07,
 5.27,
 7.2,
 5.86,
 5.71,
 5.4,
 3.47,
 5.53,
 5.93,
 5.13,
 6.0,
 6.47,
 5.47,
 5.85,
 5.73,
 6.0,
 6.86,
 6.4,
 5.93,
 7.07,
 5.8,
 6.4,
 5.67,
 6.47,
 6.07,
 5.6,
 5.6,
 5.2,
 5.57,
 5.71,
 4.67,
 6.67,
 6.0,
 5.29,
 7.0,
 4.43,
 5.53,
 6.2,
 6.73,
 6.67,
 5.93,
 6.4,
 4.47,
 5.4,
 5.15,
 5.47,
 5.47,
 5.47,
 6.13,
 5.57,
 6.14,
 5.86,
 5.29,
 5.29,
 6.43,
 6.07,
 5.87,
 4.33,
 5.07,
 6.6,
 6.0,
 5.2,
 5.93,
 6.0,
 6.07,
 6.07,
 5.93,
 5.27,
 5.2,
 6.2,
 6.07,
 5.2,
 4.2,
 6.07,
 6.33,
 6.07,
 6.85,
 5.64,
 5.67,
 5.93,
 6.8,
 3.86,
 5.53,
 6.8,
 5.43,
 5.87,
 5.87,
 4.57,
 5.4,
 7.14,
 6.2,
 6.08,
 5.67,
 5.0,
 7.07,
 5.57,
 5.93,
 5.5,
 4.0,
 6.0,
 4.33,
 5.27,
 5.93,
 5.0,
 5.33,
 6.0,
 4.07,
 5.67,
 6.27,
 6.27,
 5.4,
 3.64,
 6.27,

In [ ]:
zero = (race_and_traits_df['Race'] == 1)
# zero

In [ ]:
race_and_traits_df.loc[zero]

In [ ]:
race_and_traits_df.loc[zero].mean()

In [ ]:
def mean_median_for_each_race(df):
    """
    """
    

    all_means = []
    all_medians = []
    race_numbers = df.loc[0:, 'Race'].unique()
    race_numbers = np.sort(race_numbers)
    
    for race_number in race_numbers:
        
        specific_race = (df['Race'] == race_number)
        specific_race_series = df.loc[specific_race]
        
        specific_race_mean_series = specific_race_series.mean()
        all_means.append(specific_race_mean_series)
        
        specific_race_median_series = specific_race_series.median()
        all_medians.append(specific_race_median_series)
        

    return all_means, all_medians

In [ ]:
# race_numbers = dataset_copy.loc[0:, 'Race'].unique()
# race_numbers = np.sort(race_numbers)
# race_numbers

In [ ]:
all_race_means, all_race_medians = mean_median_for_each_race(race_and_traits_df)    

## Mean

In [ ]:
race_traits_columns = list(race_and_traits_df.columns)
race_mean_df = pd.DataFrame(columns=race_traits_columns)
# race_mean_df

In [ ]:
race_mean_df.append(all_race_means, ignore_index=True)

## Median

In [ ]:
race_median_df = pd.DataFrame(columns=race_traits_columns)
# race_median_df

In [ ]:
race_median_df.append(all_race_medians)

## 2. T-Test

- To compare 11 traits across 2 genders
- IV: gender [male, female]
- DV: psychological traits = [confident, egotistic, intelligent, kind, responsible, trustworthy, aggressive, caring, emotional, friendly, sociable]
- Outcome: p-value for each trait that'll represent the bias...

In [9]:
gender_series = dataset_copy.loc[0:, 'Gender']
# gender_series

In [10]:
gender_and_traits_df = pd.concat([gender_series, traits_df], axis=1)
gender_and_traits_df

,Gender,Confident,Egotistic,Intelligent,Kind,Responsible,Trustworthy,Aggressive,Caring,Emotional,Friendly,Sociable
0,1,5.67,3.67,5.60,5.00,5.80,5.33,4.80,4.67,3.67,4.33,4.67
1,0,6.53,2.20,6.60,7.00,7.47,6.73,3.13,6.93,5.80,6.73,6.60
2,1,6.07,3.60,6.47,6.47,6.67,6.40,2.40,6.07,5.14,6.67,6.67
3,1,4.80,4.60,4.47,3.80,4.27,3.87,5.80,3.53,2.80,2.93,3.27
4,1,6.67,5.40,6.33,4.87,5.87,4.67,4.27,4.07,4.80,5.20,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...
2217,0,6.40,3.73,6.27,5.53,5.87,6.07,3.87,6.13,4.33,5.53,5.27
2218,0,5.47,5.73,5.67,4.40,5.67,4.07,5.33,4.60,4.67,4.40,4.13
2219,1,6.67,4.20,5.60,5.93,5.67,5.53,3.80,5.87,4.80,6.53,6.87
2220,0,6.27,3.93,6.07,5.93,5.60,6.60,3.80,5.53,5.07,6.67,6.13


In [11]:
filt_gender_0 = (gender_and_traits_df['Gender'] == 0)
# filt_gender_0

In [12]:
# filt_race = (df['Race'] == race_number)
female_df = gender_and_traits_df.loc[filt_gender_0]
female_df

,Gender,Confident,Egotistic,Intelligent,Kind,Responsible,Trustworthy,Aggressive,Caring,Emotional,Friendly,Sociable
1,0,6.53,2.20,6.60,7.00,7.47,6.73,3.13,6.93,5.80,6.73,6.60
5,0,6.50,2.36,6.00,6.64,6.36,6.86,2.14,6.21,5.64,6.43,6.50
18,0,6.87,3.40,6.53,6.20,6.40,6.47,3.80,6.67,5.87,6.73,6.40
22,0,7.13,3.20,6.27,6.73,6.53,6.73,2.07,6.93,5.80,7.27,6.47
23,0,6.20,3.87,5.73,6.20,5.73,5.93,2.93,6.20,5.20,6.20,6.73
...,...,...,...,...,...,...,...,...,...,...,...,...
2215,0,5.87,4.47,5.67,5.67,5.80,5.53,2.73,6.07,5.93,5.93,6.27
2217,0,6.40,3.73,6.27,5.53,5.87,6.07,3.87,6.13,4.33,5.53,5.27
2218,0,5.47,5.73,5.67,4.40,5.67,4.07,5.33,4.60,4.67,4.40,4.13
2220,0,6.27,3.93,6.07,5.93,5.60,6.60,3.80,5.53,5.07,6.67,6.13


In [13]:
filt_gender_1 = (gender_and_traits_df['Gender'] == 1)
# filt_gender_1

In [15]:
male_df = gender_and_traits_df.loc[filt_gender_1]
male_df

,Gender,Confident,Egotistic,Intelligent,Kind,Responsible,Trustworthy,Aggressive,Caring,Emotional,Friendly,Sociable
0,1,5.67,3.67,5.60,5.00,5.80,5.33,4.80,4.67,3.67,4.33,4.67
2,1,6.07,3.60,6.47,6.47,6.67,6.40,2.40,6.07,5.14,6.67,6.67
3,1,4.80,4.60,4.47,3.80,4.27,3.87,5.80,3.53,2.80,2.93,3.27
4,1,6.67,5.40,6.33,4.87,5.87,4.67,4.27,4.07,4.80,5.20,5.33
6,1,6.20,2.27,5.13,6.60,5.87,6.33,2.87,5.93,5.20,7.07,6.07
...,...,...,...,...,...,...,...,...,...,...,...,...
2212,1,4.40,5.47,3.87,4.00,3.20,4.20,4.87,4.67,5.60,4.93,4.93
2213,1,5.79,3.57,6.79,5.79,6.50,6.79,3.57,5.71,5.29,5.93,5.57
2214,1,5.21,4.86,5.43,4.07,4.57,4.00,5.50,4.31,3.64,4.14,4.57
2216,1,5.20,3.33,6.27,6.60,5.67,5.93,3.53,5.00,4.47,5.73,5.60


In [35]:
# list(female_df.loc[0:, 'Confident'])

In [36]:
# list(male_df.loc[0:, 'Confident'])

In [91]:

def female_male_ttest(f_df, m_df):
    """
    """
    
    significant_traits = []
    significant_p_values = []
    store_p_values_in_df = pd.DataFrame()
    for trait in range(len(traits)):
        specific_trait = traits[trait]
        # print(specific_trait)
        
        # print(f_df[specific_trait], m_df[specific_trait])
        stat_result, p_result = stats.ttest_ind(f_df.loc[0:, specific_trait], m_df.loc[0:, specific_trait])
        # print(stat_result)
        # print(p_result)
        
        if p_result < 0.01:
            # print(specific_trait, p_result, '**')
            significant_traits.append(specific_trait)
            significant_p_values.append(p_result)
        elif p_result < 0.05:
            pass
            # print(specific_trait, p_result, '*')
        else:
            pass
            # print(specific_trait, p_result)

    return significant_traits, significant_p_values

In [97]:
significant_traits, significant_p_values = female_male_ttest(female_df, male_df)
significant_traits, significant_p_values

(['Confident',
  'Egotistic',
  'Intelligent',
  'Kind',
  'Responsible',
  'Trustworthy',
  'Aggressive',
  'Caring',
  'Emotional',
  'Friendly',
  'Sociable'],
 [0.0002490671279198179,
  5.724535735483598e-103,
  6.617541566954897e-10,
  3.6772754632964984e-84,
  5.314276336798082e-34,
  5.706506511591532e-73,
  6.29311355462135e-101,
  6.745136923994714e-105,
  1.029377924430242e-164,
  3.7108186188491455e-64,
  3.1946748840509533e-62])

In [105]:
store_significant_traits_with_value = {}
for significant_traits_idx in range(len(significant_traits)):
    trait = significant_traits[significant_traits_idx]
    store_significant_traits_with_value[trait] = significant_p_values[significant_traits_idx]

In [106]:
store_significant_traits_with_value

{'Confident': 0.0002490671279198179,
 'Egotistic': 5.724535735483598e-103,
 'Intelligent': 6.617541566954897e-10,
 'Kind': 3.6772754632964984e-84,
 'Responsible': 5.314276336798082e-34,
 'Trustworthy': 5.706506511591532e-73,
 'Aggressive': 6.29311355462135e-101,
 'Caring': 6.745136923994714e-105,
 'Emotional': 1.029377924430242e-164,
 'Friendly': 3.7108186188491455e-64,
 'Sociable': 3.1946748840509533e-62}